## Public Demo of IncQuery Server Jupyter Client Extensions

### Setup IQS Connection

Execute the following piece of code to connect to the public IncQuery Server demo instance.

If you have non-guest privileges, uncomment the additional lines to specify your credentials.

In [ ]:
import iqs_jupyter
iqs = iqs_jupyter.connect(
    #address='https://openmbee.incquery.io/api',
    #user='guest',
    #password='incqueryserverguest'
)

### Select MMS commit to consider

Run the next code block to display the commit selector widget, and use it to browse around the MMS repository. When you've had your fun, make sure to leave it in a state where a commit is selected from the _IQS4MMS Demos_ org, as we have made sure to pre-index and load those commits in the IQS.

In [ ]:
commit_selector = iqs.jupyter_tools.mms_commit_selector_widget()

The following piece of code assigns the Python name `model` to the MMS commit selected above, and checks whether the model is indeed indexed and loaded in memory, which is required for the rest of the demo 

In [ ]:
model = commit_selector.value().to_model_compartment()
if model.is_loaded_in_memory(iqs):
    print("We may proceed.")
else: 
    print("Model is not indexed&loaded on IQS, so the next demo steps will not work.")
    print(" (Unfortunately, guest users are not allowed to control model indexing.)")
    print("Please select another model from the 'IQS4MMS Demos' org.")

### Perform custom validation checks (progress with indexed models only)

In [ ]:
iqs.validation.validate_model_compartment(model)

### Execute pre-registered query and process results

See which queries are registered and ready for execution:

In [ ]:
iqs.queries.list_queries()

Execute a custom query registered on the server and see its results.

In [ ]:
qResults = iqs.query_execution.execute_query_on_model_compartment({
  "modelCompartment": model,
  "queryFQN": "iqs4mms.queries.example.dependencies"
})
qResults

The results can also be simply processed in Python:

In [ ]:
qResults.to_list_of_matches()

### Extract individual model elements, execute queries with parameter bindings

Descriptors of individual model elements can be extracted into Python variables from query results...

In [ ]:
first_result_element = qResults.to_list_of_matches()[0]['source']
first_result_element

...alternatively, model element descriptors can be directly constructed using element identifiers:

In [ ]:
some_element = model.get_element_in_compartment_by_id("_19_0_1_8760276_1525256443785_858460_4599") 
some_element

Model elements can be used as parameter bindings to restrict the requested results:

In [ ]:
from iqs_jupyter import binding
qResults_restricted = iqs.query_execution.execute_query_on_model_compartment({
  "modelCompartment": model,
  "queryFQN": "iqs4mms.queries.example.dependencies",
  "parameterBinding": binding(target=some_element)
})
qResults_restricted

### Convert query results to Python/Jupyter-friendly Pandas dataframes and process them as such

The following block query renders results in Pandas dataframe format, ready for complex client-side manipulation:

In [ ]:
qResults_df = qResults.to_data_frame()
qResults_df

Setting up Cufflinks for Pandas/Plot.ly visualization:

In [ ]:
import cufflinks as cf
cf.go_offline()

Preprocess results using Pandas and visualize the output using Plot.ly:

In [ ]:
qResults_df.applymap(lambda element: element.relative_element_id).source.value_counts().iplot(
    kind='bar', filename='cufflinks/categorical-bar-chart/frequent_sources',
    yTitle='Number of Dependencies', title='Dependencies from Source',
)

### Sandbox

## Extra section for privileged users
Do not forget to specify your privileged credentials at the top of the notebook, in the first code cell

### Repository management, indexing commits

Force the server to refresh its knowledge of commits in the repository:

In [ ]:
iqs.mms_repository.update_mms_repository()

Index another model from the repository, and then load the index into memory:

In [ ]:
iqs.persistent_index.index_model_compartment(model)

In [ ]:
iqs.in_memory_index.load_model_compartment(model)

### Register custom ad-hoc queries

In [ ]:
if "iqs4mms.queries.example.dependencies" not in iqs.queries.list_queries().query_fq_ns:
    # attempt to register query only if not already registered
    iqs.queries.register_queries_plain_text('''
pattern dependencies(source : NamedElement, target : NamedElement) {
  Dependency.supplier(dependency, target); Dependency.client(dependency, source);
}
''', query_package="iqs4mms.queries.example")
else:
    print("Query is already registered")    

### Sandbox